In [22]:
import os
import pandas as pd
import numpy as np
from tqdm import *

In [23]:
allImages = os.listdir('../BigDataSource/Teddy2024/附件2/ImageData')
allImages

['Image14001012-7288.jpg',
 'Image14001010-9209.jpg',
 'Image14001007-7424.jpg',
 'Image14001009-5164.jpg',
 'Image14001007-8717.jpg',
 'Image14001012-1821.jpg',
 'Image14001011-5341.jpg',
 'Image14001010-1655.jpg',
 'Image14001008-8332.jpg',
 'Image14001008-7001.jpg',
 'Image14001009-3515.jpg',
 'Image14001013-2482.jpg',
 'Image14001010-8117.jpg',
 'Image14001012-6196.jpg',
 'Image14001011-3730.jpg',
 'Image14001007-9409.jpg',
 'Image14001010-7224.jpg',
 'Image14001009-8440.jpg',
 'Image14001011-6890.jpg',
 'Image14001009-7773.jpg',
 'Image14001011-8665.jpg',
 'Image14001008-4508.jpg',
 'Image14001011-7556.jpg',
 'Image14001009-1302.jpg',
 'Image14001011-1127.jpg',
 'Image14001010-5433.jpg',
 'Image14001013-0295.jpg',
 'Image14001012-4781.jpg',
 'Image14001011-1641.jpg',
 'Image14001010-5355.jpg',
 'Image14001009-9038.jpg',
 'Image14001008-5170.jpg',
 'Image14001012-3788.jpg',
 'Image14001009-1464.jpg',
 'Image14001013-6182.jpg',
 'Image14001012-2496.jpg',
 'Image14001011-8103.jpg',
 

In [24]:
allImagesList = pd.read_csv('../BigDataSource/Teddy2024/附件2/image_data.csv')
allImagesList.shape

(50000, 1)

In [25]:
import chardet
from modules.models.SBERT import Text2Vec

file_name = '../BigDataSource/Teddy2024/附件2/word_test.csv'
# 首先，读取文件的一部分字节来检测编码
with open(file_name, 'rb') as f:
    raw_data = f.read(5000)
    result = chardet.detect(raw_data)
    encoding = result['encoding']
allTextList = pd.read_csv(file_name, encoding=encoding)
allTextList = np.array(allTextList)

text2vec = Text2Vec()
for i in trange(len(allTextList)):
    allTextList[i, 1] = text2vec.text2vec(allTextList[i, 1])
print('Done!')

100%|██████████| 5000/5000 [00:53<00:00, 93.15it/s] 

Done!


In [26]:
# allImagesFeatures = []
# for i in trange(len(allImages)):
#     imageTranfer = Image2tensor()
#     file_name = '../BigDataSource/Teddy2024/附件2/ImageData/' + allImages[i]
#     image = Image.open(file_name)
#     features = imageTranfer.image2tensor(image)
#     allImagesFeatures.append(features)
# allImagesFeatures

In [27]:
from modules.models.ResNet50 import Image2tensor
from PIL import Image

# 这里是执行多线程代码，线性运算过慢，线性预算代码在上面，用时8mins
from concurrent.futures import ThreadPoolExecutor, as_completed

# function
def process_image(inputs):
    imageTranfer, image_name = inputs
    file_name = '../BigDataSource/Teddy2024/附件2/ImageData/' + image_name
    image = Image.open(file_name)
    features = imageTranfer.image2tensor(image)
    return features

# input
allImages = os.listdir('../BigDataSource/Teddy2024/附件2/ImageData')
imageTranfer = Image2tensor()
inputList = [(imageTranfer, imageNames) for imageNames in allImages]

# Execute
allImagesFeatures = []
allImages = os.listdir('../BigDataSource/Teddy2024/附件2/ImageData')
with ThreadPoolExecutor(max_workers=16) as executor:
    futures = [executor.submit(process_image, inputs) for inputs in inputList]
    for future in tqdm(as_completed(futures), total=len(allImages)):
        allImagesFeatures.append(future.result())
print('Done!')

 91%|█████████▏| 45701/50000 [18:25<02:01, 35.28it/s]/Users/zengyuxiang/opt/anaconda3/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:792: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 4. 
  warnings.warn(str(msg))
100%|██████████| 50000/50000 [20:23<00:00, 40.85it/s]

Done!


In [39]:
import pickle
with open('../BigDataSource/Teddy2024/附件2/image_features.pkl', 'wb') as f:
    pickle.dump(allImagesFeatures, f)

In [28]:
# 将所有变量转成numpy形式
allImagesFeatures_now = []
for i in range(len(allImagesFeatures)):
    imageFeatures = np.array(allImagesFeatures[i]).reshape(-1, )
    allImagesFeatures_now.append(imageFeatures)
allImagesFeatures = np.array(allImagesFeatures_now)
allImagesFeatures.shape

(50000, 1000)

In [29]:
# 将所有变量转成numpy形式
allTextFeatures_now = []
for i in range(len(allTextList)):
    textFeatures = np.array(allTextList[i, 1]).reshape(-1, )
    allTextFeatures_now.append(textFeatures)
allTextFeatures = np.array(allTextFeatures_now)
allTextFeatures.shape

(5000, 384)

In [30]:
# from modules.models.Retrieve import LSH, L2Index
# from time import time 
# 
# d = 64  # 向量维度
# nb = 190000  # 数据库大小
# nq = 256 * 10  # 查询向量的数量
# np.random.seed(1234)  # 保证可复现的结果
# 
# # 生成一些随机样本向量
# xb = np.random.random((nb, d))
# xq = np.random.random((nq, d))
# 
# t1 = time()
# lsh = LSH(k=3, d=d, nbits=d)
# topk_distence, topk_indices = lsh.search_topk_embeds(xb, xq)
# t2 = time()
# print(f'LSH: {t2 - t1 : .2f}s')
# 
# t1 = time()
# l2index = L2Index(k=3, d=d)
# topk_distence, topk_indices = l2index.search_topk_embeds(xb, xq)
# t2 = time()
# print(f'L2: {t2 - t1 : .2f}s')

In [31]:
allTextFeatures.shape, allImagesFeatures.shape

((5000, 384), (50000, 1000))

In [32]:
from sklearn.decomposition import PCA
import numpy as np

# 假设 `visual_features` 是一个形状为 (n_samples, 1000) 的数组，

# 初始化PCA对象，设置目标维度为384
pca = PCA(n_components=384)

# 对视觉特征进行降维
scaled_allImagesFeatures = pca.fit_transform(allImagesFeatures)

# 查看降维后的形状，确认是否为 (n_samples, 384)
print("降维后的视觉特征形状:", scaled_allImagesFeatures.shape)


降维后的视觉特征形状: (50000, 384)


In [33]:
xb = scaled_allImagesFeatures
xq = allTextFeatures

In [34]:
from modules.models.Retrieve import LSH, L2Index
from time import time 

d = 384  # 向量维度
k = 5
np.random.seed(1234)  # 保证可复现的结果

t1 = time()
lsh = LSH(k=k, d=d, nbits=4)
topk_distence, topk_indices = lsh.search_topk_embeds(xb, xq)
t2 = time()
print(f'LSH: {t2 - t1 : .2f}s')


LSH:  0.05s


In [35]:
topk_indices

array([[ 19,   1,  37,  22,  18],
       [ 19,   1,  37,  22,  18],
       [ 66,  29,  92,  84,  25],
       ...,
       [  8,   6,  47,  24,   5],
       [ 69,  42, 157,  93,  20],
       [ 62,  53,  82,  75,  33]])

In [36]:
xb = scaled_allImagesFeatures
xq = allTextFeatures

In [37]:

t1 = time()
l2index = L2Index(k=k, d=d)
topk_distence, topk_indices = l2index.search_topk_embeds(xb, xq)
t2 = time()
print(f'L2: {t2 - t1 : .2f}s')

L2:  0.63s


In [38]:
topk_indices

array([[49495, 22639, 49241, 11832,  1771],
       [49495, 22639, 49241, 11832,  1771],
       [49495, 22639, 49241, 11832,  1771],
       ...,
       [49495, 49241, 22639, 11832,  1771],
       [49495, 22639, 49241, 11832,  1771],
       [49495, 22639, 49241,  1771, 11832]])